In [ ]:
import tensorflow as tf
import os
import time
import math
import numpy as np
import re
import time
import csv
import warnings
import threading
warnings.filterwarnings('ignore', '.*do not.*',)
#input: A Tensor. Must be one of the following types: half, bfloat16, float32, float64. 
#Shape [batch, in_depth, in_height, in_width, in_channels].

batch_size_input= 2
learning_rate_input=0.0001 #학습률
grad_clip_input=5 #do not touch
decrease_learing_rate=0.8 #매 반복마다 학습률 감소비율
num_epochs_input=1205     
dataset_input=0

output_number_input=9 # 질문이 있냐 없냐므로 2개
output_temp=list()
dropout_rate_input=0.1 #드랍아웃 확률임 
width_input=80     #영상 가로 픽셀 개수
lenth_input=40     #영상 세로 픽셀 개수
depth_input=3      #하나의 픽셀에 차원 개수 열영상은 온도 1개므로 1차원, 일반 카메라는 픽셀당 RGB 3개값이므로 3차원
file_input='.csv'
restore_='result_test.csv'
with tf.device('/cpu:0'):
    class VideoDataset():
        def __init__(self, filenames, timesteps=10):
        
            self.set_timesteps(timesteps)
            self.temp_red=list()
            self.temp_blue=list()
            self.temp_green=list()
            xs_0=list()
            ys_0=list()
            xs_0_=list()
            ys_0_temp=list()
            xs_0_temp=list()
            #print(xs_0)
            self.tr_xs = np.arange(36).reshape((2,3,3,2))
            print("종우",self.tr_xs.shape)
            self.tr_ys = np.arange(40,290,1).reshape((2,5,5,5))

        def set_timesteps(self, timesteps):
            self.timesteps = timesteps
            self.input_shape = [timesteps+1, 100, 200, 3] #이미지 개수
            self.output_shape = [timesteps+1, 2] #출력 2개의 속력

        @property
        def train_count(self):
            return len(self.tr_xs)

        
        def get_train_data(self, batch_size, nth):
            
            
            return self.tr_xs, self.tr_ys

        def shuffle_train_data(self, size):
            np.random.seed(int(time.time()))
            self.indices = np.arange(size)
            np.random.shuffle(self.indices)

        def get_test_data(self):
            #print("the number of data is ", self.te_xs.shape)
            return self.te_xs, self.te_ys


        def divide_data(self, xs, ys, tr_ratio=0.7):
            data_count = len(xs)

            tr_cnt = int(data_count * tr_ratio / 10) * 10
            te_cnt = data_count - (tr_cnt)

            tr_from, tr_to = 0, tr_cnt
            te_from, te_to = tr_cnt, data_count
            print("divide_data,data_count,tr_from,tr_to,te_to",data_count,tr_from,tr_to,te_to)
            self.tr_xs = xs[tr_from:tr_to]
            self.tr_ys = ys[tr_from:tr_to]
            self.te_xs = xs[te_from:te_to]
            self.te_ys = ys[te_from:te_to]
            #print(self.tr_xs, self.te_xs)
            self.input_shape = xs[0].shape
            self.output_shape = ys[0].shape
        
        
    

        
        
    class ConvNN(object):
        def __init__(self, batchsize=batch_size_input,
                 epochs=num_epochs_input, learning_rate=learning_rate_input, 
                 dropout_rate=dropout_rate_input, width=width_input, lenth=lenth_input, depth=depth_input,
                 output_number=output_number_input,dataset=dataset_input,
                  random_seed=None):
            np.random.seed(int(time.time()))
            self.batchsize = batchsize
            self.epochs = epochs
            self.learning_rate = learning_rate
            self.dropout_rate = dropout_rate
            self.dataset=dataset
            self.width=width_input
            self.length=lenth_input
            self.depth=depth_input
            self.output_number= output_number
                
            g = tf.Graph()
            with g.as_default():
            ## set random-seed:
                tf.set_random_seed(random_seed)
            
            ## build the network:
                self.build()

            ## initializer
                self.init_op = tf.global_variables_initializer()

            ## saver
                self.saver = tf.train.Saver()
            
        ## create a session
            self.sess = tf.Session(graph=g)
        def batch_generator(self,predict=False): #데이터 전처리용


           
            
            return self.dataset.tr_xs,self.dataset.tr_ys

        def target_batch_generator(self,predict=True):
            X=self.dataset.te_xs
            Y=self.dataset.te_ys
            return X, Y       
        def build(self):
        
        ## Placeholders for X and y:
            tf_x = tf.placeholder(tf.float32, 
                              shape=[2,3,3,2],
                              name='tf_x')
            tf_y = tf.placeholder(tf.float32, 
                              shape=[2,5,5,5],
                              name='tf_y')
            is_train = tf.placeholder(tf.bool, 
                              shape=(),
                              name='is_train')
        
            #print(tf_x)
            #print(tf_y)

            self.h1 = tf.layers.conv2d_transpose(inputs=tf_x,filters=5,
                              kernel_size=(3, 3), strides=(1,1) ,
                               kernel_initializer=tf.random_normal_initializer(mean=0,stddev=0.03),
                              activation=None,name='h1')
            
            #print(self.h1.eval())
         
        
        
        ## Loss Function and Optimization
            cost =tf.reduce_mean(tf.losses.mean_squared_error(
                 labels=tf_y, predictions=self.h1), name='cost')
            print(cost)
        
        ## Optimizer:
            optimizer = tf.train.AdamOptimizer(self.learning_rate)
            train_op = optimizer.minimize(cost,name='train_op')
            print(train_op)
       

        def save(self, epoch, path='./tflayers-model/'):
            if not os.path.isdir(path):
                os.makedirs(path)
            print('Saving model in %s' % path)
            self.saver.save(self.sess, 
                        os.path.join(path, 'model.ckpt'),
                        global_step=epoch)
        
        def load(self, epoch, path):
            print('Loading model from %s' % path)
            self.saver.restore(self.sess, 
             os.path.join(path, 'model.ckpt-%d' % epoch))
        
        def train(self, ckpt_dir='model_2/', 
              initialize=True):
        ## initialize variables
           
            ckpt_dir='./'+ckpt_dir+'/'
            start = time.time()
            if not os.path.exists(ckpt_dir):
                os.mkdir(ckpt_dir)
            if initialize:
                self.sess.run(self.init_op)
            

            for epoch in range(self.epochs):
             
                batch_x,batch_y = self.batch_generator()
  
                self.learning_rate=decrease_learing_rate*self.learning_rate
                
                avg_loss = 0.0
                print(batch_x.shape)
                feed = {'tf_x:0': batch_x, 
                    'tf_y:0': batch_y,
                    'is_train:0': True} ## for dropout
                output_=self.sess.run(['h1'], 
                    feed_dict={'tf_x:0': batch_x})
                print(output_.eval())
                loss, _ = self.sess.run(
                    ['cost:0', 'train_op'], 
                    feed_dict=feed)
                print(self.h1.eval())
                print(batch_x)
                self.saver.save(self.sess,ckpt_dir+"image-%d.ckpt" % epoch)
                with open(restore_,'a',newline='') as aa:
                    writer = csv.writer(aa, delimiter=',')
                    avg_loss = loss
                    ttt=self.predict_in()
                    writer.writerow([epoch+1]+[i]+[np.mean(avg_loss)]+[ttt])

                print('epoch ',epoch,'i ' ,i,'avg_loss ', avg_loss,"test_loss",ttt)
       
            
                    
        def predict(self, X_test, return_proba = False):
            feed = {'tf_x:0': X_test,
                'is_train:0': False} ## for dropout
            if return_proba:
                return self.sess.run('probabilities:0',
                                 feed_dict=feed)
            else:
                return self.sess.run('labels:0',
                                 feed_dict=feed)
        
        def predict_in(self, ckpt='./f_ma_original_/',initialize=False):
    
            
            self.saver.restore(self.sess, tf.train.latest_checkpoint(ckpt))
                
                
            batch_x, batch_y = self.target_batch_generator(predict=True)
            pred_list=list()
            batch_y_list=list()
            
                        
                  
            feed = {'tf_x:0': batch_x, 
                    'tf_y:0': batch_y,
                    'is_train:0': False} ## for dropout
            pred = self.sess.run([self.h5],feed_dict=feed)

            
                
            diff = batch_y- pred
            #print("diff.shape",diff.shape)
            square = np.square(diff)
            loss = np.mean(square[:,:])    
            return loss
        
  
     

In [ ]:
with tf.device('/cpu:0'):
    vsd  = VideoDataset(['physical_intution'])


In [ ]:
with tf.device('/cpu:0'):
    
   
    cnn=ConvNN(dataset=vsd) #CNN 모델 클래스 하나 생성

    cnn.train(ckpt_dir='f_ma_original_') #학습시켜서 ckpt_dir 에다가 기록해라